In [58]:
from pathlib import Path
from typing import List, Dict

from loguru import logger
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col, when
import sys
import geopy.distance

In [2]:
CONF_LOG_PREFIX = 'CONFLOG'
FLST_LOG_PREFIX = 'FLSTLOG'
GEO_LOG_PREFIX = 'GEOLOG'
LOS_LOG_PREFIX = 'LOSLOG'
REG_LOG_PREFIX = 'REGLOG'
LOADING_PATH = 'C:/Users/jpedrero/SpyderProjects/M2_data_analysis_platform/output'
DATAFRAMES_NAMES = [CONF_LOG_PREFIX, FLST_LOG_PREFIX, GEO_LOG_PREFIX, LOS_LOG_PREFIX, REG_LOG_PREFIX]

In [39]:
sys.path.append(str(Path(Path().absolute().parent, 'platform_code')))
from schemas.tables_attributes import *

In [3]:
def load_dataframes(files_names: List[str], loading_path: str, spark: SparkSession) -> Dict[str, DataFrame]:
    """ Loads the dataframes which macht the file names passed by arguments.
    The method read from the config the path were to read the files, which
    matches the folder where the files are saved in `save_dataframes_dict()`.

    :param files_names: list of the names of the files.
    :param loading_path: path were the files are saved.
    :param spark: spark session.
    :return: dictionary with the dataframes loaded from the files, with the
     file name as key.
    """
    dataframes = dict()

    for file_name in files_names:
        file_path = Path(loading_path, f'{file_name.lower()}.parquet')
        logger.info('Loading dataframe from `{}`.', file_path)
        df = spark.read.parquet(str(file_path))
        dataframes[file_name] = df

    return dataframes

In [4]:
spark = SparkSession.builder.appName('Notebook').getOrCreate()

In [5]:
input_dataframes = load_dataframes(DATAFRAMES_NAMES, LOADING_PATH, spark)

2022-03-23 16:23:10.877 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `C:\Users\jpedrero\SpyderProjects\M2_data_analysis_platform\output\conflog.parquet`.
2022-03-23 16:23:42.871 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `C:\Users\jpedrero\SpyderProjects\M2_data_analysis_platform\output\flstlog.parquet`.
2022-03-23 16:23:43.332 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `C:\Users\jpedrero\SpyderProjects\M2_data_analysis_platform\output\geolog.parquet`.
2022-03-23 16:23:43.581 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `C:\Users\jpedrero\SpyderProjects\M2_data_analysis_platform\output\loslog.parquet`.
2022-03-23 16:23:43.911 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `C:\Users\jpedrero\SpyderProjects\M2_data_analysis_platform\output\reglog.parquet`.


In [10]:
input_dataframes

{'CONFLOG': DataFrame[CONF_ID: bigint, Scenario: string, CONF_detected_time: double, CPALAT: double, CPALON: double],
 'FLSTLOG': DataFrame[Flight_id: bigint, Scenario: string, ACID: string, Origin_LAT: string, Origin_LON: string, Destination_LAT: string, Destination_LON: string, Baseline_departure_time: double, cruising_speed: string, Priority: int, loitering: boolean, Baseline_2D_distance: double, Baseline_vertical_distance: double, Baseline_ascending_distance: double, Baseline_3D_distance: double, Baseline_flight_time: double, Baseline_arrival_time: double, Deletion_Time: double, Spawn_Time: double, Flight_time: string, Distance_2D: string, Distance_3D: double, Distance_ALT: double, Deletion_LAT: double, Deletion_LON: double, Deletion_ALT: double, Ascend_dist: double, Work_Done: double],
 'GEOLOG': DataFrame[GEO_id: bigint, Scenario: string, Deletion_Time: double, Geofence_name: string, Max_intrusion: double, Violation_severity: boolean, Open_airspace: boolean, Loitering_nfz: boolea

In [85]:
df1 = input_dataframes[FLST_LOG_PREFIX].withColumn("sum1", col(SPAWN_TIME) - col(BASELINE_DEPARTURE_TIME)).groupby(SCENARIO_NAME, PRIORITY).agg(F.sum("sum1").alias("sum1"))
df1.show()

+------------------+--------+---------+
|          Scenario|Priority|     sum1|
+------------------+--------+---------+
|1_very_low_40_8_R2|       4|   4810.0|
|1_very_low_40_8_R2|       1|1782582.0|
|1_very_low_40_8_W1|       4|   4810.0|
|1_very_low_40_8_W1|       2|1861986.0|
|1_very_low_40_8_R2|       2|1861986.0|
|1_very_low_40_8_W1|       3|1840296.0|
|1_very_low_40_8_W1|       1|1782582.0|
|1_very_low_40_8_R2|       3|1840296.0|
|3_very_low_40_8_R2|       1|1782582.0|
|3_very_low_40_8_W1|       2|1861986.0|
|3_very_low_40_8_R2|       4|   4810.0|
|3_very_low_40_8_W1|       4|   4810.0|
|3_very_low_40_8_W1|       3|1840296.0|
|3_very_low_40_8_W1|       1|1782582.0|
|3_very_low_40_8_R2|       2|1861986.0|
|3_very_low_40_8_R2|       3|1840296.0|
|2_very_low_40_8_R2|       4|   4810.0|
|2_very_low_40_8_W1|       3|1840296.0|
|2_very_low_40_8_R2|       3|1840296.0|
|2_very_low_40_8_W1|       4|   4810.0|
+------------------+--------+---------+
only showing top 20 rows



In [78]:
@udf
def distCoords(origin_LAT, origin_LON, destination_LAT, destination_LON): #coords_1=(x1,y1) and coords_2=(x2,y2)
    origin_tuple = (origin_LAT, origin_LON)
    destination_tuple = (destination_LAT, destination_LON)
    dst = geopy.distance.distance(origin_tuple, destination_tuple).m #TODO: direct distance calculation (in meters) between two points, is this approach correct?
    return dst

In [79]:
input_dataframes[FLST_LOG_PREFIX] = input_dataframes[FLST_LOG_PREFIX].withColumn(BASELINE_2D_DISTANCE, distCoords(col(ORIGIN_LAT), col(ORIGIN_LON), col(DESTINATION_LAT), col(DESTINATION_LON)))


In [80]:
input_dataframes[FLST_LOG_PREFIX] = input_dataframes[FLST_LOG_PREFIX].withColumn(BASELINE_FLIGHT_TIME, 
           when(col(LOITERING), col(BASELINE_2D_DISTANCE)/col(CRUISING_SPEED) + 60/5)
            .otherwise(col(BASELINE_2D_DISTANCE)/col(CRUISING_SPEED)))


In [84]:
df2 = input_dataframes[FLST_LOG_PREFIX].withColumn("sum2", col(FLIGHT_TIME) - col(BASELINE_FLIGHT_TIME)).groupby(SCENARIO_NAME, PRIORITY).agg(F.sum("sum2").alias("sum2"))
df2.show()


+------------------+--------+------------------+
|          Scenario|Priority|              sum2|
+------------------+--------+------------------+
|1_very_low_40_8_R2|       4| 5515.258711855475|
|1_very_low_40_8_R2|       1|1995769.9029709946|
|1_very_low_40_8_W1|       4| 5515.258711855475|
|1_very_low_40_8_W1|       2| 2087582.502777938|
|1_very_low_40_8_R2|       2| 2087582.502777938|
|1_very_low_40_8_W1|       3| 2049311.763952924|
|1_very_low_40_8_W1|       1|1995769.9029709946|
|1_very_low_40_8_R2|       3| 2049311.763952924|
|3_very_low_40_8_R2|       1|1995769.9029709946|
|3_very_low_40_8_W1|       2| 2087582.502777938|
|3_very_low_40_8_R2|       4| 5515.258711855475|
|3_very_low_40_8_W1|       4| 5515.258711855475|
|3_very_low_40_8_W1|       3| 2049311.763952924|
|3_very_low_40_8_W1|       1|1995769.9029709946|
|3_very_low_40_8_R2|       2| 2087582.502777938|
|3_very_low_40_8_R2|       3| 2049311.763952924|
|2_very_low_40_8_R2|       4| 5515.258711855475|
|2_very_low_40_8_W1|

In [82]:
df1.show()

+------------------+--------+---------+
|          Scenario|Priority|sum(sum1)|
+------------------+--------+---------+
|1_very_low_40_8_R2|       4|   4810.0|
|1_very_low_40_8_R2|       1|1782582.0|
|1_very_low_40_8_W1|       4|   4810.0|
|1_very_low_40_8_W1|       2|1861986.0|
|1_very_low_40_8_R2|       2|1861986.0|
|1_very_low_40_8_W1|       3|1840296.0|
|1_very_low_40_8_W1|       1|1782582.0|
|1_very_low_40_8_R2|       3|1840296.0|
|3_very_low_40_8_R2|       1|1782582.0|
|3_very_low_40_8_W1|       2|1861986.0|
|3_very_low_40_8_R2|       4|   4810.0|
|3_very_low_40_8_W1|       4|   4810.0|
|3_very_low_40_8_W1|       3|1840296.0|
|3_very_low_40_8_W1|       1|1782582.0|
|3_very_low_40_8_R2|       2|1861986.0|
|3_very_low_40_8_R2|       3|1840296.0|
|2_very_low_40_8_R2|       4|   4810.0|
|2_very_low_40_8_W1|       3|1840296.0|
|2_very_low_40_8_R2|       3|1840296.0|
|2_very_low_40_8_W1|       4|   4810.0|
+------------------+--------+---------+
only showing top 20 rows



In [86]:
df3 = df1.join(df2, on=[SCENARIO_NAME, PRIORITY], how='outer')
df3.show()

+------------------+--------+---------+------------------+
|          Scenario|Priority|     sum1|              sum2|
+------------------+--------+---------+------------------+
|1_very_low_40_8_R2|       1|1782582.0|1995769.9029709946|
|1_very_low_40_8_R2|       2|1861986.0| 2087582.502777938|
|1_very_low_40_8_R2|       3|1840296.0| 2049311.763952924|
|1_very_low_40_8_R2|       4|   4810.0| 5515.258711855475|
|1_very_low_40_8_W1|       1|1782582.0|1995769.9029709946|
|1_very_low_40_8_W1|       2|1861986.0| 2087582.502777938|
|1_very_low_40_8_W1|       3|1840296.0| 2049311.763952924|
|1_very_low_40_8_W1|       4|   4810.0| 5515.258711855475|
|2_very_low_40_8_R2|       1|1782582.0|1995769.9029709946|
|2_very_low_40_8_R2|       2|1861986.0| 2087582.502777938|
|2_very_low_40_8_R2|       3|1840296.0| 2049311.763952924|
|2_very_low_40_8_R2|       4|   4810.0| 5515.258711855475|
|2_very_low_40_8_W1|       1|1782582.0|1995769.9029709946|
|2_very_low_40_8_W1|       2|1861986.0| 2087582.50277793

In [96]:
df = df3.withColumn(PRI5, col("sum1") + col("sum2")).select(SCENARIO_NAME, PRIORITY, PRI5)
df.show()

+------------------+--------+------------------+
|          Scenario|Priority|              PRI5|
+------------------+--------+------------------+
|1_very_low_40_8_R2|       1| 3778351.902970995|
|1_very_low_40_8_R2|       2| 3949568.502777938|
|1_very_low_40_8_R2|       3| 3889607.763952924|
|1_very_low_40_8_R2|       4|10325.258711855475|
|1_very_low_40_8_W1|       1| 3778351.902970995|
|1_very_low_40_8_W1|       2| 3949568.502777938|
|1_very_low_40_8_W1|       3| 3889607.763952924|
|1_very_low_40_8_W1|       4|10325.258711855475|
|2_very_low_40_8_R2|       1| 3778351.902970995|
|2_very_low_40_8_R2|       2| 3949568.502777938|
|2_very_low_40_8_R2|       3| 3889607.763952924|
|2_very_low_40_8_R2|       4|10325.258711855475|
|2_very_low_40_8_W1|       1| 3778351.902970995|
|2_very_low_40_8_W1|       2| 3949568.502777938|
|2_very_low_40_8_W1|       3| 3889607.763952924|
|2_very_low_40_8_W1|       4|10325.258711855475|
|3_very_low_40_8_R2|       1| 3778351.902970995|
|3_very_low_40_8_R2|

In [65]:
input_dataframes[FLST_LOG_PREFIX].select(BASELINE_FLIGHT_TIME, BASELINE_2D_DISTANCE).show()

+--------------------+--------------------+
|Baseline_flight_time|Baseline_2D_distance|
+--------------------+--------------------+
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|                 0.0|
|                 0.0|          

In [56]:
df2 = input_dataframes[FLST_LOG_PREFIX].withColumn("sum2",  col(FLIGHT_TIME) - col(BASELINE_FLIGHT_TIME)).groupby("Scenario", "Priority").agg(F.sum("sum2")).show(50)


+------------------+--------+-------------------+
|          Scenario|Priority|          sum(sum2)|
+------------------+--------+-------------------+
|1_very_low_40_8_R2|       4|-28515.343855446205|
|1_very_low_40_8_R2|       1| -5316737.302873654|
|1_very_low_40_8_W1|       4|-28515.343855446205|
|1_very_low_40_8_W1|       2| -5657315.973030068|
|1_very_low_40_8_R2|       2| -5657315.973030068|
|1_very_low_40_8_W1|       3| -5488091.352724353|
|1_very_low_40_8_W1|       1| -5316737.302873654|
|1_very_low_40_8_R2|       3| -5488091.352724353|
|3_very_low_40_8_R2|       1| -5316737.302873654|
|3_very_low_40_8_W1|       2| -5657315.973030068|
|3_very_low_40_8_R2|       4|-28515.343855446205|
|3_very_low_40_8_W1|       4|-28515.343855446205|
|3_very_low_40_8_W1|       3| -5488091.352724353|
|3_very_low_40_8_W1|       1| -5316737.302873654|
|3_very_low_40_8_R2|       2| -5657315.973030068|
|3_very_low_40_8_R2|       3| -5488091.352724353|
|2_very_low_40_8_R2|       4|-28515.343855446205|


In [22]:
for d in df1:
    print(d)

Column<'Scenario'>
Column<'ACID'>
Column<'Spawn_Time'>
Column<'Baseline_departure_time'>
